## Phase 7 – Build Gradio App for Prediction + Explanation

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from tensorflow.keras.applications.vgg16 import preprocess_input


In [ ]:
# Step 2 — Load Model and Define Preprocessing

In [ ]:
# Load your trained model
model_path = "/content/drive/MyDrive/AML2_Project_Models/cnn_model.h5"
model = load_model(model_path)
print("✅ Model loaded successfully!")

# Define class names (update based on your dataset)
class_names = [
    'Actinic Keratoses',        # Class 0
    'Basal Cell Carcinoma',     # Class 1
    'Benign Keratosis',         # Class 2
    'Dermatofibroma',           # Class 3
    'Melanoma',                 # Class 4
    'Nevus'                     # Class 5
]

# Function to preprocess input image
def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array


In [4]:
# Step 3 — Define Prediction and Explanation Function

In [ ]:
def predict_and_explain(img):
    # Preprocess
    img_array = preprocess_image(img)

    # Prediction
    preds = model.predict(img_array)
    pred_class = np.argmax(preds, axis=1)[0]
    confidence = float(np.max(preds))

    # LIME Explanation
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(
        img_array[0].astype('double'),
        model.predict,
        top_labels=1,
        hide_color=0,
        num_samples=1000
    )

    temp, mask = explanation.get_image_and_mask(
        label=pred_class,
        positive_only=True,
        hide_rest=False,
        num_features=5,
        min_weight=0.05
    )

    lime_img = mark_boundaries(temp / 255.0, mask)

    # Create readable output
    result = {class_names[i]: float(preds[0][i]) for i in range(len(class_names))}
    result_text = f"Predicted: {class_names[pred_class]} (Confidence: {confidence:.2f})"

    return lime_img, result_text, result


In [5]:
# Step 4 — Build the Gradio Interface

In [ ]:
# Create Gradio interface
interface = gr.Interface(
    fn=predict_and_explain,
    inputs=gr.Image(type="pil", label="Upload a Skin Lesion Image"),
    outputs=[
        gr.Image(type="numpy", label="LIME Explanation"),
        gr.Textbox(label="Prediction Summary"),
        gr.Label(label="Class Probabilities")
    ],
    title="🩺 Skin Lesion Classification with Explainable AI",
    description="Upload an image to view the model prediction and LIME-based explanation."
)

# Launch app
interface.launch(share=True)
